# Introduction to Spark

## how to use markdown
https://grabngoinfo.com/databricks-notebook-markdown-cheat-sheet/

A **spark session** is not necessarly needed in databricks

Using a builder design pattern, it instantiates a SparkSession object if one does not already exist, along with its associated underlying contexts.

Otherwise, we should create a SparkSession in your notebook python / scala / R

To prouve this, we can create a new databrick notebook (new session) and execute "spark"

In [ ]:
from os.path import abspath
from pyspark.sql import SparkSession

# warehouse_location
warehouse_location = abspath('spark-warehouse')

# Create spark session with hive enabled
spark = (
    SparkSession 
    .builder
    .appName("SparkByExamples.com")
#     .config("spark.sql.warehouse.dir", warehouse_location)
    .enableHiveSupport()
    .getOrCreate()
)


### 1. Dataframe and partitions

In [ ]:
# myRange = spark.range(1000).toDF("number") # toDF is just for renaming
myRange = spark.range(1000) # is already a DF, 
myRange.rdd.getNumPartitions() # get the number of partitions

myRange.show(5)
myRange.display()
type(myRange)

+---+
| id|
+---+
|  0|
|  1|
|  2|
|  3|
|  4|
+---+
only showing top 5 rows



id 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233 234 235 236 237 238 239 240 241 242 243 244 245 246 247 248 249 250 251 252 253 254 255 256 257 258 259 260 261 262 263 264 265 266 267 268 269 270 271 272 273 274 275 276 277 278 279 280 281 282 283 284 285 286 287 288 289 290 291 292 293 294 295 296 297 298 299 300 301 302 303 304 305 306 307 308 309 310 311 312 313 314 315 316 317 318 319 320 321 322 323 324 325 326 327 328 329 330 331 332 333 334 335 336 337 338 339 340 341 342 343 344 345 346 347 348 349 350 351 352 353 354 355 356 357 358 359 360 361 362 363 364 365 366 367 368 369 370 371 372 373 374 375 376 377 378 379 380 381 382 383 384 385 386 387 388 389 390 391 392 393 394 395 396 397 398 399 400 401 402 403 404 405 406 407 408 409 410 411 412 413 414 415 416 417 418 419 420 421 422 423 424 425 426 427 428 429 430 431 432 433 434 435 436 437 438 439 440 441 442 443 444 445 446 447 448 449 450 451 452 453 454 455 456 457 458 459 460 461 462 463 464 465 466 467 468 469 470 471 472 473 474 475 476 477 478 479 480 481 482 483 484 485 486 487 488 489 490 491 492 493 494 495 496 497 498 499 500 501 502 503 504 505 506 507 508 509 510 511 512 513 514 515 516 517 518 519 520 521 522 523 524 525 526 527 528 529 530 531 532 533 534 535 536 537 538 539 540 541 542 543 544 545 546 547 548 549 550 551 552 553 554 555 556 557 558 559 560 561 562 563 564 565 566 567 568 569 570 571 572 573 574 575 576 577 578 579 580 581 582 583 584 585 586 587 588 589 590 591 592 593 594 595 596 597 598 599 600 601 602 603 604 605 606 607 608 609 610 611 612 613 614 615 616 617 618 619 620 621 622 623 624 625 626 627 628 629 630 631 632 633 634 635 636 637 638 639 640 641 642 643 644 645 646 647 648 649 650 651 652 653 654 655 656 657 658 659 660 661 662 663 664 665 666 667 668 669 670 671 672 673 674 675 676 677 678 679 680 681 682 683 684 685 686 687 688 689 690 691 692 693 694 695 696 697 698 699 700 701 702 703 704 705 706 707 708 709 710 711 712 713 714 715 716 717 718 719 720 721 722 723 724 725 726 727 728 729 730 731 732 733 734 735 736 737 738 739 740 741 742 743 744 745 746 747 748 749 750 751 752 753 754 755 756 757 758 759 760 761 762 763 764 765 766 767 768 769 770 771 772 773 774 775 776 777 778 779 780 781 782 783 784 785 786 787 788 789 790 791 792 793 794 795 796 797 798 799 800 801 802 803 804 805 806 807 808 809 810 811 812 813 814 815 816 817 818 819 820 821 822 823 824 825 826 827 828 829 830 831 832 833 834 835 836 837 838 839 840 841 842 843 844 845 846 847 848 849 850 851 852 853 854 855 856 857 858 859 860 861 862 863 864 865 866 867 868 869 870 871 872 873 874 875 876 877 878 879 880 881 882 883 884 885 886 887 888 889 890 891 892 893 894 895 896 897 898 899 900 901 902 903 904 905 906 907 908 909 910 911 912 913 914 915 916 917 918 919 920 921 922 923 924 925 926 927 928 929 930 931 932 933 934 935 936 937 938 939 940 941 942 943 944 945 946 947 948 949 950 951 952 953 954 955 956 957 958 959 960 961 962 963 964 965 966 967 968 969 970 971 972 973 974 975 976 977 978 979 980 981 982 983 984 985 986 987 988 989 990 991 992 993 994 995 996 997 998 999

Out[4]: pyspark.sql.dataframe.DataFrame

You just ran your first Spark code! We created a DataFrame with one column containing 1,000 rows with values from 0 to 999. This range of numbers represents a distributed collection. When run on a cluster, each part of this range of numbers exists on a different executor. This is a Spark DataFrame.

### 2. Transformations & Action
Expliquer la notion de Transformation vs Action

In [ ]:
myRange.where("id % 2 = 0")# filter sur les nombres paires
divisBy2 = myRange.where("id % 2 = 0") 
divisBy2

# why there is no result => because this is a transformation (transformation is abstract), non execution sur data

divisBy2.show()

# Here we have a output, Spark will not act on transformations until we call an action, who triggers the transformation, il will declenche execution of myRange then filter (see explain())

divisBy2.explain()

# You can read explain plans from top to bottom, the top being the end result, and the bottom being the source(s) of data. 

+---+
| id|
+---+
|  0|
|  2|
|  4|
|  6|
|  8|
| 10|
| 12|
| 14|
| 16|
| 18|
| 20|
| 22|
| 24|
| 26|
| 28|
| 30|
| 32|
| 34|
| 36|
| 38|
+---+
only showing top 20 rows

== Physical Plan ==
*(1) Filter ((id#135L % 2) = 0)
+- *(1) Range (0, 1000, step=1, splits=8)




Transformation est une opération Spark qui lit un DataFrame (je prends df comme exemple), manipule des colonnes et éventuellement renvoie un autre DataFrame. Les exemples de transformation comprennent les opérations courantes : filtre, sélection, tri, groupby, map, join [écrire au tableau]…

Narrow transformations : chaque partition d'entrée ne contribue qu'à une seule partition de sortie.

Wide transformation : Les partitions doivent être mélangée pour agrégée pour obtenir un résultat

Dans wide transformation il y a forcément Shuffle Operations
Shuffle Operations
Une Shuffle Operations est déclenchée lorsque des données doivent être mélangées donc déplacées entre les exécuteurs (Spark va échanger des partitions à travers le cluster). Il s'agit d'un élément essentiel de nombreuses transformations, telles que .groupBy(), et de certaines actions, telles que .count().

In [ ]:
# Both df have the same number of partitions, why? Because we did a filter => introduit narrow transformations and wide transformations
myRange.rdd.getNumPartitions()

divisBy2.rdd.getNumPartitions()

Out[9]: 8

### 3. THEN  
#### 3.1 lazy evaluation

Maintenant on va voir pour quoi transformation ne donne pas d'output => ne déclenche pas d'éxecution

Les transformations sont évaluées en mode Lazy Evaluation. Cela signifie qu'aucun job Spark n'est déclenché par les transformations, quel que soit le nombre de transformations programmées. Pas de job=> pas d’exécution.

Lazy Evaluation est un modèle courant dans les langages fonctionnels et de Big Data, par exemple Spark, Scala, Java Stream API, etc. 

Si pas d’exécution, que fait-elle une transformation ? Elle construit un plan de logique d’exécution, si vous enchainer les transformations, le plan logique va être enrichi. Si nous construisons un gros travail Spark (join, map…) [schemas au tableau] mais que nous spécifions un filtre à la fin qui ne nécessite que l'extraction de quelques lignes uniquement, la manière la plus efficace d'exécuter ce travail est de faire le filter au début. Spark optimisera cela pour nous en poussant le filtre au début (vers le bas sur le plan) automatiquement.

Au lieu de modifier les données immédiatement, Spark attendra le dernier moment pour exécuter son plan logique. Il va attendre une action qui déclenche l’exécution. Les avantages sont immenses car Spark peut optimiser l'ensemble du flux de données.

D'ailleurs, les utilisateurs peuvent coder des petites transformations (exemple : enchaine plusieurs filters) pour faciliter la lecture et la gestion. Mais Spark regroupe en interne ces transformations, en réduisant le nombre de passages sur les données. Si Spark pouvait regrouper deux transformations en une seule, il n'aurait à lire les données qu'une seule fois pour appliquer les transformations, au lieu de les lire deux fois .


#### 3.2 Action

Les transformations construit un plan de modification de DataFrame. Plus précisement, les transformations nous permettent d'élaborer un plan logique. Pas d’exécution réelle. Elles ne sont exécutées que lorsqu'une action est appelée sur un DataFrame.

Pour déclencher le calcul des transformations, nous appelons une action : une opération Spark qui renvoie un résultat ou écrit sur le disque.
- .show() : afficher les données dans la console
- .count()
- .take() : renvoie le nombre spécifié d'enregistrements au driver process
- .collect() : collecte tous les résultats de tous les nœuds de travail et les renvoie au driver process. N'utilisez cette méthode que pour renvoyer de petites données agrégées.
- .toPandas() : collecte tous les enregistrements de tous les travailleurs, les renvoie au driver process, puis convertit les résultats en un DataFrame pandas. N'utilisez cette méthode que pour renvoyer de petites données agrégées.
- .head(), .first(), .last()

#### 3.3 Overview of Structured API Execution
Voici un aperçu des étapes à suivre :
1. Écrire le DataFrame/Dataset/ SQL Code.
2. Si le code est valide, Spark le convertit en plan logique.
3. Spark transforme ce plan logique en plan physique, en vérifiant les optimisations en cours de route.
4. Spark exécute ensuite ce Plan Physique (manipulations RDD) sur le cluster.

#### 3.4 Logical plan
1. Convertir un code utilisateur vers un plan non résolu parce que même si votre code est valide (verification de syntaxe : parenthèses, nom de function...) les tables ou les colonnes auxquelles il se réfère peuvent exister ou ne pas exister. 
2. Spark utilise catalog, un référentiel de toutes les informations sur les tables et les DataFrame, pour résoudre les colonnes et les tables dans l'analyseur.
3. Le résultat résolu passe ensuite par l'Optimiseur Catalyst (filtre à la fin...).<br/>
=> A la fin vous aurez un plan logique validé et optimisé

#### 3.5 Physical plan
Après avoir créé un plan logique optimisé, Spark entame l’élaboration du plan physique. Le plan physique, souvent appelé plan Spark, spécifie comment le plan logique sera exécuté concrètement sur le cluster en générant différentes stratégies d'exécution physique. Puis spark va les comparer à l'aide d'un modèle de coût (cost model). Physical plan est réellement une série de RDD et de transformations. C’est la raison pour laquelle vous avez peut-être entendu parler de Spark comme un compilateur - il prend des requêtes (= transformations) de DataFrames, Datasets et SQL et les compile de des transformations RDD, et surtout il effectue énormément d’optimisations avant (et durant ??) cette compilation grâce à Catalyst Optimizer et cost model.

#### 3.6 Execution

Une fois un plan physique optimal sélectionné, Spark exécute tous ces codes de transformations sur des RDD (lower-level programming interface). Spark effectue des optimisations supplémentaires au moment de l'exécution, en générant du native Java bytecode qui peut supprimer des tâches ou des étapes entières au cours de l'exécution. Enfin, le résultat est renvoyé à l'utilisateur.

### An End-to-End Example to "*remettre une couche*": transformation (narrow or wide), action, partitions, lazy evaluation
In the previous example, we created a DataFrame of a range of numbers; not exactly groundbreaking big data. here we will reinforce everything we learned previously in
this chapter with a more realistic example

In [ ]:
retrail =(spark.read
# .option("inferSchema", "true")
.option("header", "true")
.csv("/databricks-datasets/definitive-guide/data/retail-data/all"))

# we can also use ()

In [ ]:
retrail.show(5)
retrail.count() #541909

+---------+---------+--------------------+--------+--------------+---------+----------+--------------+
|InvoiceNo|StockCode|         Description|Quantity|   InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+--------------------+--------+--------------+---------+----------+--------------+
|   536365|   85123A|WHITE HANGING HEA...|       6|12/1/2010 8:26|     2.55|     17850|United Kingdom|
|   536365|    71053| WHITE METAL LANTERN|       6|12/1/2010 8:26|     3.39|     17850|United Kingdom|
|   536365|   84406B|CREAM CUPID HEART...|       8|12/1/2010 8:26|     2.75|     17850|United Kingdom|
|   536365|   84029G|KNITTED UNION FLA...|       6|12/1/2010 8:26|     3.39|     17850|United Kingdom|
|   536365|   84029E|RED WOOLLY HOTTIE...|       6|12/1/2010 8:26|     3.39|     17850|United Kingdom|
+---------+---------+--------------------+--------+--------------+---------+----------+--------------+
only showing top 5 rows

Out[16]: 541909

In [ ]:
retrail.where("Quantity <= 6").rdd.getNumPartitions() # 8
retrail.where("Quantity <= 6").explain() # just FileScan


df = retrail.collect()
pd_df = retrail.where("Quantity <= 6").toPandas()

type(df)

type(pd_df)

Out[10]: pandas.core.frame.DataFrame

In [ ]:
retrail.sort("UnitPrice").explain()

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- Sort [UnitPrice#178 ASC NULLS FIRST], true, 0
   +- Exchange rangepartitioning(UnitPrice#178 ASC NULLS FIRST, 200), ENSURE_REQUIREMENTS, [plan_id=288]
      +- FileScan csv [InvoiceNo#173,StockCode#174,Description#175,Quantity#176,InvoiceDate#177,UnitPrice#178,CustomerID#179,Country#180] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex(1 paths)[dbfs:/databricks-datasets/definitive-guide/data/retail-data/all], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<InvoiceNo:string,StockCode:string,Description:string,Quantity:string,InvoiceDate:string,Un...




In [ ]:
retrail.rdd.getNumPartitions()

Out[18]: 8

In [ ]:
# spark.conf.set("spark.sql.shuffle.partitions", "12") # this doesn't work seems that databrick do not accept partitions set
retrail.where("Quantity <= 6").sort("UnitPrice").rdd.getNumPartitions()


retrail.where("Quantity <= 6").sort("UnitPrice").explain()

Out[19]: 9

In specifying this action, we started a Spark job that runs our filter transformation (a narrow transformation), then an aggregation (a wide transformation) that performs the sort on a per partition basis, and then a collect, which brings our result to a native object in the respective language.

You can see all of this by inspecting the Spark UI, a tool included in Spark with which you can monitor the Spark jobs running on a cluster.

 You can read explain plans from top to bottom, the top being the end result, and the bottom being the source(s) of data. You will see sort, exchange, filter and FileScan. That’s because the sort of our data is actually a wide transformation because rows will need to be compared with one another.

### 4. Job, stages and tasks

In [ ]:
df1 = spark.range(2, 100000, 2) # => stage 1, with 8 Tasks
df2 = spark.range(2, 100000, 4) # => stage 2, with 8 Tasks

step1 = df1.repartition(5) # => stage 3, with 5 Tasks

step12 = df2.repartition(6) # => stage 4, with 6 Tasks

step2 = step1.selectExpr("id * 5 as id") 
step3 = step2.join(step12, ["id"]) # => 2 instructions as stage 5, with 8 Tasks (by default)
step4 = step3.selectExpr("sum(id)") # => stage 6, with 1 Tasks because we aggregate only one result

step4.collect()

step4.rdd.getNumPartitions()

step4.explain()

Out[32]: 1

### 5. Spark UI